# Eager Execution
- Here TensorFlow operations are executed by Python, operation by operation, and returning results back to Python.
- TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well. 
- To follow along with this guide, run the code samples below in an interactive `python` interpreter.
- reference: https://www.tensorflow.org/guide/eager

## Basics

### Basic Import

In [3]:
import os
import tensorflow as tf
import cProfile

### enable eagerly execution
Enabling eager execution changes how TensorFlow operations behave—now they immediately evaluate and return their values to Python. `tf.Tensor` objects reference concrete values instead of symbolic handles to nodes in a computational graph. Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using `print()` or a debugger. Evaluating, printing, and checking tensor values does not break the flow for computing gradients.

In [4]:
tf.executing_eagerly()

True

## Operation

### multiplication

In [8]:
x = [[2.]]
x

[[2.0]]

In [9]:
m = tf.matmul(x,x)

In [10]:
print("hello, {}".format(m))

hello, [[4.]]


### addition

In [11]:
a = tf.constant([[1,2],
                [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [12]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


### Dynamic control flow
A major benefit of eager execution is that all the functionality of the host language is available while your model is executing. So, for example, it is easy to write fizzbuzz:



In [13]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    max_num = tf.convert_to_tensor(max_num)
    
    for num in range(1, max_num.numpy()+1):
        num = tf.constant(num)
        if int(num%3) ==0 and int(num%5) == 0:
            print('FizzBuzz')
        elif int(num %3) ==0:
            print('Fuzz')
        elif int(num %5)==0:
            print('Buzz')
        else:
            print(num.numpy())
        counter +=1

In [14]:
fizzbuzz(15)

1
2
Fuzz
4
Buzz
Fuzz
7
8
Fuzz
Buzz
11
Fuzz
13
14
FizzBuzz


# tf.functions
`tf.function` constructs a `tf.types.experimental.GenericFunction` that executes a TensorFlow graph (`tf.Graph`) created by trace-compiling the TensorFlow operations in func.

In [15]:
@tf.function
def f(x,y):
    return x ** 2 +y 

In [16]:
x = tf.constant([2,3])
y = tf.constant([3, -2])
f(x, y)

2021-09-30 11:42:52.683138: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 7], dtype=int32)>

# Graph
- While eager execution has several unique advantages, graph execution enables portability outside Python and tends to offer better performance. Graph execution means that tensor computations are executed as a TensorFlow graph, sometimes referred to as a `tf.Graph` or simply a `graph`.
- Graphs are data structures that contain a set of `tf.Operation` objects, which represent units of computation; and `tf.Tensor` objects, which represent the units of data that flow between operations. They are defined in a `tf.Graph` context. Since these graphs are data structures, they can be saved, run, and restored all without the original Python code.
- Tensorflow graph representating a 2layers neural network look like when visualized in tensorboard:
<div>
<img src="input_images/graph_2layer_network.jpg" width="700" height="700" style="float:left"/>
</div>
- reference: https://www.tensorflow.org/guide/intro_to_graphs


## Benefits
- With a graph, you have a great deal of flexibility. You can use your TensorFlow graph in environments that don't have a Python interpreter, like mobile applications, embedded devices, and backend servers. TensorFlow uses graphs as the format for saved models when it exports them from Python.
- Graphs are also easily optimized, allowing the compiler to do transformations like:
    - Statically infer the value of tensors by folding constant nodes in your computation ("constant folding").
    - Separate sub-parts of a computation that are independent and split them between threads or devices.
    - Simplify arithmetic operations by eliminating common subexpressions.
- There is an entire optimization system, Grappler, to perform this and other speedups.
- In short, graphs are extremely useful and let your TensorFlow run fast, run in parallel, and run efficiently on multiple devices. However, you still want to define your machine learning models (or other computations) in Python for convenience, and then automatically construct graphs when you need them.







In [19]:
## Deinf a Python function
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x +b
    return x

In [20]:
# 'a_function_that_use_a_graph' is a Tensorflow Function
a_function_that_use_a_graph = tf.function(a_regular_function)

In [22]:
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[[2.0], [3.0]]])
b1 = tf.constant(4.0)

In [23]:
original_value = a_regular_function(x1, y1, b1).numpy()

In [24]:
tf_function_value = a_function_that_use_a_graph(x1, y1, b1).numpy()

In [26]:
## On the outside, a Function looks like a regular function you write using TensorFlow operations. 
## Underneath, however, it is very different. 
## A Function encapsulates several tf.Graphs behind one API. That is how Function is able to give you the 
## benefits of graph execution, like speed and deployability.

assert(original_value==tf_function_value)

In [27]:
tf_function_value

array([[[12.]]], dtype=float32)

In [28]:
original_value

array([[[12.]]], dtype=float32)

# Graph execution vs Eager Execution
The code in a `Function` can be executed both eagerly and as a graph. By default, `Function` executes its code as a graph:

In [40]:
## Eagerly mode
tf.config.run_functions_eagerly(False)

In [41]:
### Graph
@tf.function
def get_mse(y_true, y_pred):
    sq_diff = tf.pow(y_true,y_pred)
    print("Calculating MSE")
    return tf.reduce_mean(sq_diff)

In [42]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

tf.Tensor([7 3 1 7 5], shape=(5,), dtype=int32)
tf.Tensor([1 3 9 4 2], shape=(5,), dtype=int32)


In [43]:
error=get_mse(y_true, y_pred)
error=get_mse(y_true, y_pred)
error=get_mse(y_true, y_pred)

Calculating MSE


### Eagerly to True
To verify that your Function's graph is doing the same computation as its equivalent Python function, you can make it execute eagerly with tf.config.run_functions_eagerly(True). This is a switch that turns off Function's ability to create and run graphs, instead executing the code normally.



In [44]:
## Eagerly mode
tf.config.run_functions_eagerly(True)

In [46]:
error=get_mse(y_true, y_pred)
error=get_mse(y_true, y_pred)
error=get_mse(y_true, y_pred)

Calculating MSE
Calculating MSE
Calculating MSE
